# Query Model Repository for best node embeddings model

In [42]:
from hops import model
from hops.model import Metric
MODEL_NAME="NodeEmbeddings"
EVALUATION_METRIC="accuracy"

In [43]:
best_model = model.get_best_model(MODEL_NAME, EVALUATION_METRIC, Metric.MAX)

In [44]:
print('Model name: ' + best_model['name'])
print('Model version: ' + str(best_model['version']))
print(best_model['metrics'])

Model name: NodeEmbeddings
Model version: 1
{'accuracy': '0.6678586006164551'}

# Create Model Serving of Exported Model

In [45]:
from hops import serving

In [46]:
MODEL_NAME

'NodeEmbeddings'

In [47]:
# Create serving
model_path="/Models/" + best_model['name']
model_path
response = serving.create_or_update(artifact_path=model_path, serving_name=MODEL_NAME, serving_type="TENSORFLOW", 
                                 model_version=best_model['version'])

Creating a serving for model NodeEmbeddings ...
Serving for model NodeEmbeddings successfully created

In [48]:
# List all available servings in the project
for s in serving.get_all():
    print(s.name)

NodeEmbeddings

In [49]:
# Get serving status
serving.get_status(MODEL_NAME)

'Running'

# Start Model Serving Server

In [50]:
if serving.get_status(MODEL_NAME) == 'Stopped':
    serving.start(MODEL_NAME)

In [51]:
import time
while serving.get_status(MODEL_NAME) != "Running":
    time.sleep(5) # Let the serving startup correctly
time.sleep(5)

# Send Prediction Requests to the Served Model using Hopsworks REST API

In [23]:
import pandas as pd
from stellargraph import StellarDiGraph
from stellargraph.mapper import Node2VecLinkGenerator
from stellargraph.data import UnsupervisedSampler
from stellargraph.data import BiasedRandomWalk

import hsfs
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

In [24]:
node_td = fs.get_training_dataset("node_td", 1)
edge_td = fs.get_training_dataset("edges_td", 1)

In [25]:
# Get fg as pandas
node_pdf = node_td.read().toPandas()
edge_pdf = edge_td.read().drop("tran_timestamp").toPandas()

In [26]:
walk_number = 2
walk_length = 2

In [35]:
node_data = pd.DataFrame(node_pdf[['tx_behavior_id','prior_sar','initial_deposit']], index=node_pdf['id'])

print('Defining StellarDiGraph')
G =StellarDiGraph(node_data,
                      edges=edge_pdf, 
                      edge_type_column="tx_type")
walker = BiasedRandomWalk(
        G,
        n=walk_number,
        length=walk_length,
        p=0.5,  # defines probability, 1/p, of returning to source node
        q=2.0,  # defines probability, 1/q, for moving to a node away from the source node
    )
unsupervised_samples = UnsupervisedSampler(G, nodes=list(G.nodes()), walker=walker)
generator = Node2VecLinkGenerator(G, 32)

Defining StellarDiGraph

In [62]:
serve_data = generator.flow(unsupervised_samples)
serve_data = serve_data._create_batches()

In [75]:
import numpy as np
TOPIC_NAME = serving.get_kafka_topic(MODEL_NAME)

import json
for i in range(1):
    data = {
                "signature_name": "serving_default", "instances": [[serve_data[0][0].tolist(), serve_data[0][1].tolist()]]
            }
    response = serving.make_inference_request(MODEL_NAME, data)
    print(response)

An error was encountered:
Could not create or update serving (url: /hopsworks-api/api/project/181/inference/models/NodeEmbeddings:predict), server response: 
 HTTP code: 400, HTTP reason: Bad Request, error code: 250008, error msg: Serving instance bad request error, user msg: {
    "error": "instances is a plain list, but expecting list of objects as multiple input tensors required as per tensorinfo_map"
}
Traceback (most recent call last):
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/hops/serving.py", line 527, in make_inference_request
    return _make_inference_request_rest(serving_name, data, verb)
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/hops/serving.py", line 562, in _make_inference_request_rest
    error_code, error_msg, user_msg))
hops.exceptions.RestAPIError: Could not create or update serving (url: /hopsworks-api/api/project/181/inference/models/NodeEmbeddings:predict), server response: 
 HTTP code: 400, HTTP reason: Bad Reques